In [1]:
from oddt.toolkits import ob
from joblib import delayed, Parallel
from functools import partial
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from matplotlib import pyplot as plt
from scipy import stats

import json
import numpy as np

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

PDBBIND_DIR = '/home/fergus/pdbbind/v2019'
with open('../data/pdbbind_training_set.txt') as f:
    pdbs = [l.strip() for l in f]
with open('../data/pdbbind_training_set.txt') as f:
    train_pdbs = [l.strip() for l in f]

In [3]:
from oddt.scoring.descriptors import close_contacts_descriptor
ligand_atomic_nums = [6, 7, 8, 9, 15, 16, 17, 35, 53]
protein_atomic_nums = [6, 7, 8, 16]
cutoff = [0, 2, 4, 6, 8, 10, 12]
descriptors_func = close_contacts_descriptor(cutoff=cutoff,protein_types=protein_atomic_nums,ligand_types=ligand_atomic_nums)

@delayed
def build(pdb):
    protein = next(ob.readfile('pdb', f'{PDBBIND_DIR}/{pdb}/{pdb}_protein.pdb'))
    protein.protein = True
    # crystal
    ligand = next(ob.readfile('sdf', f'{PDBBIND_DIR}/{pdb}/{pdb}_ligand.sdf'))
    features_c = descriptors_func.build(ligand, protein)[0]

    # docked        
    ligand = next(ob.readfile('sdf', f'../pdbbind_docked_poses/{pdb}/{pdb}_ligand_docked.sdf'))
    features_d = descriptors_func.build(ligand, protein)[0]
    return (features_c, features_d)

In [8]:
with Parallel(n_jobs=6, verbose=10) as parallel:
    features = parallel(build(pdb) for pdb in pdbs)

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    1.7s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    2.5s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:    4.3s
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:   10.3s
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:   15.3s
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:   21.1s
[Parallel(n_jobs=6)]: Done  49 tasks      | elapsed:   31.0s
[Parallel(n_jobs=6)]: Done  60 tasks      | elapsed:   32.8s
[Parallel(n_jobs=6)]: Done  73 tasks      | elapsed:   35.8s
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:   40.9s
[Parallel(n_jobs=6)]: Done 101 tasks      | elapsed:   49.0s
[Parallel(n_jobs=6)]: Done 116 tasks      | elapsed:   56.9s
[Parallel(n_jobs=6)]: Done 133 tasks      | elapsed:  1.0min
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed:  1.2min
[Parallel(n_jobs=6)]: Done 169 tasks      | elapsed:  1.5min
[Parallel(

In [12]:
features_crystal = {}
features_docked = {}

for feature, pdb in zip(features, pdbs):
    features_crystal[pdb] = feature[0]
    features_docked[pdb] = feature[1]

In [14]:
with open('pdbbind_training_set_docked_rfv2.json', 'w') as f:
    json.dump(features_docked, f, cls=NumpyEncoder)
    
with open('pdbbind_training_set_crystal_rfv2.json', 'w') as f:
    json.dump(features_crystal, f, cls=NumpyEncoder)

In [2]:
with open('../data/pdbbind_test_set.txt') as f:
    test_pdbs = [l.strip() for l in f]

In [38]:
with Parallel(n_jobs=6, verbose=10) as parallel:
    test_features = parallel(build(pdb) for pdb in test_pdbs)

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


NameError: name 'build' is not defined

In [23]:
test_features_crystal = {}
test_features_docked = {}

for feature, pdb in zip(test_features, test_pdbs):
    test_features_crystal[pdb] = feature[0]
    test_features_docked[pdb] = feature[1]
    
with open('pdbbind_test_set_docked_rfv2.json', 'w') as f:
    json.dump(test_features_docked, f, cls=NumpyEncoder)
    
with open('pdbbind_test_set_crystal_rfv2.json', 'w') as f:
    json.dump(test_features_crystal, f, cls=NumpyEncoder)

In [3]:
pdbbind_training_set_affinity = pd.read_csv('../data/pdbbind_training_set_binding_affinity.csv', index_col=0, header=None, squeeze=True)
pdbbind_test_set_affinity = pd.read_csv('../data/pdbbind_test_set_binding_affinity.csv', index_col=0, header=None, squeeze=True)

In [8]:
features_docked = pd.Series(features_docked)

In [4]:
with open('pdbbind_training_set_docked_rfv2.json') as f:
    features_docked = json.load(f)
features_docked = pd.DataFrame(features_docked).T
with open('pdbbind_training_set_crystal_rfv2.json') as f:
    features_crystal = json.load(f)
features_crystal = pd.DataFrame(features_crystal).T

In [87]:
a=pd.DataFrame({'Crystal': {'RFv2': 0.760, 'RFv2 + Ligand': 0.787}, 'Docked': {'RFv2': 0.723, 'RFv2 + Ligand': 0.768}}).T

In [88]:
b=pd.DataFrame({'Crystal': {'RFv3': 0.765, 'RFv3 + Ligand': 0.785}, 'Docked': {'RFv3': 0.692, 'RFv3 + Ligand': 0.756}}).T

In [91]:
c=pd.DataFrame({'Crystal': {'PLEC': 0.689}, 'Docked': {'PLEC': 0.645}}).T

In [93]:
pd.concat([a,b,c], axis='columns')

,RFv2,RFv2 + Ligand,RFv3,RFv3 + Ligand,PLEC
Crystal,0.760,0.787,0.765,0.785,0.689
Docked,0.723,0.768,0.692,0.756,0.645


In [5]:
with open('pdbbind_test_set_crystal_rfv2.json') as f:
    test_features_crystal = pd.DataFrame(json.load(f)).T    
with open('pdbbind_test_set_docked_rfv2.json') as f:
    test_features_docked = pd.DataFrame(json.load(f)).T

In [6]:
pdbbind_training_set_affinity = pd.read_csv('../data/pdbbind_training_set_binding_affinity.csv', index_col=0, header=None, squeeze=True)
pdbbind_test_set_affinity = pd.read_csv('../data/pdbbind_test_set_binding_affinity.csv', index_col=0, header=None, squeeze=True)

In [7]:
pdbbind_training_set = pdbbind_training_set_affinity.index
pdbbind_test_set = pdbbind_test_set_affinity.index

In [8]:
blast_clusters = {}
for cutoff in [30, 40, 50, 70, 90, 95, 100]:
    with open(f'../data/bc-{cutoff}.out') as f:
        blast_clusters[cutoff] = [set(item[:4].lower() for item in line.strip().split()) for line in f]

test_set_similar_pdbs = {}
for cutoff in blast_clusters:
    pdbs = set()
    for pdb in pdbbind_test_set:
        for cluster in blast_clusters[cutoff]:
            if pdb in cluster:
                pdbs.update(cluster)
    test_set_similar_pdbs[cutoff] = pd.Index(pdbs).intersection(pdbbind_training_set)

test_set_similar_pdbs['All'] = pd.Index([])

In [9]:
with open('../data/pdbbind_ligand_tanimoto_similarity.json') as f:
    tanimoto_similarity = json.load(f)

tc_thresholds = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

test_set_similar_ligands = {}

for t in tc_thresholds:
    similar = set()
    for pdb_test in pdbbind_test_set:
        for pdb_train in pdbbind_training_set:
            if pdb_train in similar:
                continue
            if tanimoto_similarity[pdb_test][pdb_train] >= t:
                similar.add(pdb_train)
    test_set_similar_ligands[t] = pd.Index(similar)
test_set_similar_ligands['All'] = pd.Index([])

In [10]:
%%time
predicted_crystal_sequence_identity = {}
predicted_docked_sequence_identity = {}
pearsonr_crystal_sequence_identity = {}
pearsonr_docked_sequence_identity = {}

for cutoff in test_set_similar_pdbs:
    print(cutoff)
    training_set = pdbbind_training_set.difference(test_set_similar_pdbs[cutoff])
    y_train = pdbbind_training_set_affinity.loc[training_set].values.ravel()
    X_train_crystal = features_crystal.loc[training_set].values
    X_train_docked = features_docked.loc[training_set].values
    X_test_crystal = test_features_crystal.loc[pdbbind_test_set].values
    X_test_docked = test_features_docked.loc[pdbbind_test_set].values
    y_test = pdbbind_test_set_affinity.loc[pdbbind_test_set].values.ravel()
    rf_crystal = RandomForestRegressor(n_estimators=500, max_features=0.33,random_state=42, n_jobs=8, oob_score=True)
    rf_crystal.fit(X_train_crystal, y_train)
    rf_docked = RandomForestRegressor(n_estimators=500, max_features=0.33,random_state=42, n_jobs=8, oob_score=True)
    rf_docked.fit(X_train_docked, y_train)
    
    predicted_crystal = rf_crystal.predict(X_test_crystal)
    predicted_docked = rf_docked.predict(X_test_docked)
    
    predicted_crystal_sequence_identity[cutoff] = {pdb: pred for pdb, pred in zip(pdbbind_test_set, predicted_crystal)}
    predicted_docked_sequence_identity[cutoff] = {pdb: pred for pdb, pred in zip(pdbbind_test_set, predicted_docked)}
    
    pearsonr_crystal_sequence_identity[cutoff] = stats.pearsonr(y_test, predicted_crystal)[0]
    pearsonr_docked_sequence_identity[cutoff] = stats.pearsonr(y_test, predicted_docked)[0]

30
40
50
70
90
95
100
All
CPU times: user 4min 23s, sys: 2.11 s, total: 4min 25s
Wall time: 47 s


In [11]:
lb_features = pd.read_csv('lb_features.csv', index_col=0)

In [12]:
%%time
predicted_crystal_rdk_sequence_identity = {}
predicted_docked_rdk_sequence_identity = {}
pearsonr_crystal_rdk_sequence_identity = {}
pearsonr_docked_rdk_sequence_identity = {}

for cutoff in test_set_similar_pdbs:
    print(cutoff)
    training_set = pdbbind_training_set.difference(test_set_similar_pdbs[cutoff])
    y_train = pdbbind_training_set_affinity.loc[training_set].values.ravel()
    X_train_crystal = features_crystal.loc[training_set]
    X_train_docked = features_docked.loc[training_set]
    X_test_crystal = test_features_crystal.loc[pdbbind_test_set]
    X_test_docked = test_features_docked.loc[pdbbind_test_set]
    y_test = pdbbind_test_set_affinity.loc[pdbbind_test_set].values.ravel()
    # add lb features
    X_train_crystal = pd.concat([X_train_crystal, lb_features.loc[training_set]], axis='columns')
    X_test_crystal = pd.concat([X_test_crystal, lb_features.loc[pdbbind_test_set]], axis='columns')
    X_train_docked = pd.concat([X_train_docked, lb_features.loc[training_set]], axis='columns')
    X_test_docked = pd.concat([X_test_docked, lb_features.loc[pdbbind_test_set]], axis='columns')
    rf_crystal = RandomForestRegressor(n_estimators=500, max_features=0.33,random_state=42, n_jobs=8, oob_score=True)
    rf_crystal.fit(X_train_crystal, y_train)
    rf_docked = RandomForestRegressor(n_estimators=500, max_features=0.33,random_state=42, n_jobs=8, oob_score=True)
    rf_docked.fit(X_train_docked, y_train)
    
    predicted_crystal = rf_crystal.predict(X_test_crystal)
    predicted_docked = rf_docked.predict(X_test_docked)
    
    predicted_crystal_rdk_sequence_identity[cutoff] = {pdb: pred for pdb, pred in zip(pdbbind_test_set, predicted_crystal)}
    predicted_docked_rdk_sequence_identity[cutoff] = {pdb: pred for pdb, pred in zip(pdbbind_test_set, predicted_docked)}
    
    pearsonr_crystal_rdk_sequence_identity[cutoff] = stats.pearsonr(y_test, predicted_crystal)[0]
    pearsonr_docked_rdk_sequence_identity[cutoff] = stats.pearsonr(y_test, predicted_docked)[0]

30
40
50
70
90
95
100
All
CPU times: user 9min 48s, sys: 1.88 s, total: 9min 49s
Wall time: 1min 31s


In [ ]:
with open('RFv2_predicted_crystal_sequence_identity.json', 'w') as f:
    json.dump(predicted_crystal_sequence_identity, f, cls=NumpyEncoder)
with open('RFv2_predicted_docked_sequence_identity.json', 'w') as f:
    json.dump(predicted_docked_sequence_identity, f, cls=NumpyEncoder)
with open('RFv2_pearsonr_crystal_sequence_identity.json', 'w') as f:
    json.dump(pearsonr_crystal_sequence_identity, f, cls=NumpyEncoder)
with open('RFv2_pearsonr_docked_sequence_identity.json', 'w') as f:
    json.dump(pearsonr_docked_sequence_identity, f, cls=NumpyEncoder)

In [13]:
with open('RFv2_RDK_predicted_crystal_sequence_identity.json', 'w') as f:
    json.dump(predicted_crystal_rdk_sequence_identity, f, cls=NumpyEncoder)
with open('RFv2_RDK_predicted_docked_sequence_identity.json', 'w') as f:
    json.dump(predicted_docked_rdk_sequence_identity, f, cls=NumpyEncoder)
with open('RFv2_RDK_pearsonr_crystal_sequence_identity.json', 'w') as f:
    json.dump(pearsonr_crystal_rdk_sequence_identity, f, cls=NumpyEncoder)
with open('RFv2_RDK_pearsonr_docked_sequence_identity.json', 'w') as f:
    json.dump(pearsonr_docked_rdk_sequence_identity, f, cls=NumpyEncoder)

In [16]:
import statistical_tests as st

pearsonr_ci_crystal_si = {}
pearsonr_ci_docked_si = {}
pearsonr_pval_crystal_si = {}
pearsonr_pval_docked_si = {}

y_test = pdbbind_test_set_affinity.loc[pdbbind_test_set]
for cutoff in predicted_crystal_sequence_identity:
    y_pred = pd.Series(predicted_crystal_sequence_identity[cutoff]).loc[pdbbind_test_set]
    pearsonr_ci_crystal_si[cutoff] = st.bootstrap_pearsonr(y_test, y_pred)
    pearsonr_ci_docked_si[cutoff] = st.bootstrap_pearsonr(y_test, y_pred)
    y_pred = pd.Series(predicted_docked_sequence_identity[cutoff]).loc[pdbbind_test_set]
    pearsonr_pval_crystal_si[cutoff] = st.permutation_pearsonr(y_test, y_pred)
    pearsonr_pval_docked_si[cutoff] = st.permutation_pearsonr(y_test, y_pred)

In [17]:
pearsonr_ci_crystal_rdk_si = {}
pearsonr_ci_docked_rdk_si = {}
pearsonr_pval_crystal_rdk_si = {}
pearsonr_pval_docked_rdk_si = {}

y_test = pdbbind_test_set_affinity.loc[pdbbind_test_set]
for cutoff in predicted_crystal_sequence_identity:
    y_pred = pd.Series(predicted_crystal_rdk_sequence_identity[cutoff]).loc[pdbbind_test_set]
    pearsonr_ci_crystal_rdk_si[cutoff] = st.bootstrap_pearsonr(y_test, y_pred)
    pearsonr_ci_docked_rdk_si[cutoff] = st.bootstrap_pearsonr(y_test, y_pred)
    y_pred = pd.Series(predicted_docked_rdk_sequence_identity[cutoff]).loc[pdbbind_test_set]
    pearsonr_pval_crystal_rdk_si[cutoff] = st.permutation_pearsonr(y_test, y_pred)
    pearsonr_pval_docked_rdk_si[cutoff] = st.permutation_pearsonr(y_test, y_pred)

In [18]:
pearsonr_ci_docked_si

{30: (0.6363352941528522, 0.5861732111540154, 0.6809833203604377),
 40: (0.655218555043996, 0.6064773217308622, 0.6983750638330051),
 50: (0.6679261930168756, 0.6200800430089926, 0.710638467171554),
 70: (0.6784759671589725, 0.6318741456936731, 0.7202703120665265),
 90: (0.6788647471573663, 0.6309934976573922, 0.7220444392238354),
 95: (0.6865974628613758, 0.6384876007145944, 0.7297136591450654),
 100: (0.6911562310376635, 0.6453616225840594, 0.7328836953444601),
 'All': (0.7595467454161737, 0.7169144171770295, 0.7963317518623201)}

In [19]:
pearsonr_ci_docked_rdk_si

{30: (0.6656586430897223, 0.6191025848988843, 0.7071141003743997),
 40: (0.6912263367490572, 0.646206767697391, 0.7308093094059908),
 50: (0.7015456794383016, 0.6571238189144453, 0.7403780434508314),
 70: (0.7129654154734528, 0.6693445091111934, 0.7509937444080956),
 90: (0.7159675362640138, 0.6721432740175579, 0.7547605783836621),
 95: (0.7141030679223483, 0.6695906378289265, 0.7531093592035177),
 100: (0.7232433218886716, 0.6803722349588313, 0.7609198558777046),
 'All': (0.7866430614997031, 0.7480014913627931, 0.819873097863047)}